<a href="https://colab.research.google.com/github/benmerrilll/country_hapiness_eda_project/blob/main/aggregation_of_hapiness_reports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aggregation of World Happiness Reports and World Development Indicators

## Part 1.0 Importing Libraries and Data
In this notebook, we will use numpy and pandas libraries to manipulate our data. Links to our data sources follow.

World Happiness Report: https://www.kaggle.com/unsdsn/world-happiness

World Development Indicators: https://databank.worldbank.org/source/World-Development-Indicators?savedlg=1&l=en#

In [213]:
# Libraries
import pandas as pd
import numpy as np 

In [214]:
# World Happiness Report Datasets
hp_2015 = pd.read_csv('2015.csv')
hp_2016 = pd.read_csv('2016.csv')
hp_2017 = pd.read_csv('2017.csv')
hp_2018 = pd.read_csv('2018.csv')
hp_2019 = pd.read_csv('2019.csv') 
hp_2015.head()

,Country,Region,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
0,Switzerland,Western Europe,1,7.587,0.03411,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2.51738
1,Iceland,Western Europe,2,7.561,0.04884,1.30232,1.40223,0.94784,0.62877,0.14145,0.43630,2.70201
2,Denmark,Western Europe,3,7.527,0.03328,1.32548,1.36058,0.87464,0.64938,0.48357,0.34139,2.49204
3,Norway,Western Europe,4,7.522,0.03880,1.45900,1.33095,0.88521,0.66973,0.36503,0.34699,2.46531
4,Canada,North America,5,7.427,0.03553,1.32629,1.32261,0.90563,0.63297,0.32957,0.45811,2.45176


In [215]:
# World Development Indicators dataset for countries from 2015-2019
wdi_data = pd.read_excel('WDI 2015-2019.xlsx')
print('Shape is', wdi_data.shape)
wdi_data.head()

Shape is (313136, 9)


,Country Name,Country Code,Series Name,Series Code,2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019]
0,United States,USA,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,100,100,..,..,..
1,United States,USA,Access to electricity (% of population),EG.ELC.ACCS.ZS,100,100,100,100,..
2,United States,USA,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,100,100,100,100,..
3,United States,USA,"Access to electricity, urban (% of urban popul...",EG.ELC.ACCS.UR.ZS,100,100,100,100,..
4,United States,USA,Account ownership at a financial institution o...,FX.OWN.TOTL.ZS,..,..,93.1222,..,..


## 1.1 Happiness Report Aggregation
We will aggregate the world happiness reports by first dropping any columns that are not represented in all 5 datasets and then looking at the 'Country or region' field to provide a uniform format and keep only countries shown in all 5 files.

In [216]:
# Let's add a year column to each file to help identify it
hp_2015['Year'] = 2015
hp_2016['Year'] = 2016
hp_2017['Year'] = 2017
hp_2018['Year'] = 2018
hp_2019['Year'] = 2019

In [217]:
# output to better understand each of our datasets
for file in hp_2015, hp_2016, hp_2017, hp_2018, hp_2019:
  print('Data in', file['Year'][0])
  print('File shape is ',file.shape)
  print(file.columns)
  print('\n')

Data in 2015
File shape is  (158, 13)
Index(['Country', 'Region', 'Happiness Rank', 'Happiness Score',
       'Standard Error', 'Economy (GDP per Capita)', 'Family',
       'Health (Life Expectancy)', 'Freedom', 'Trust (Government Corruption)',
       'Generosity', 'Dystopia Residual', 'Year'],
      dtype='object')


Data in 2016
File shape is  (157, 14)
Index(['Country', 'Region', 'Happiness Rank', 'Happiness Score',
       'Lower Confidence Interval', 'Upper Confidence Interval',
       'Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)',
       'Freedom', 'Trust (Government Corruption)', 'Generosity',
       'Dystopia Residual', 'Year'],
      dtype='object')


Data in 2017
File shape is  (155, 13)
Index(['Country', 'Happiness.Rank', 'Happiness.Score', 'Whisker.high',
       'Whisker.low', 'Economy..GDP.per.Capita.', 'Family',
       'Health..Life.Expectancy.', 'Freedom', 'Generosity',
       'Trust..Government.Corruption.', 'Dystopia.Residual', 'Year'],
      dtype='o

There's a few things we realize here:
- The number of countries (rows) is different for every year
- The columns lables are formatted differently

Next, we'll **reformat the column labels** manually, creating a uniform 10 fields for each year's dataframe. We will match 2015, 2016, and 2017 to the format of 2018 and 2019. The uniform columns allow us to aggregate each year's happiness report into a single dataframe.

In [218]:
# 2015
column_reorder_2015 = ['Happiness Rank', 'Country', 'Happiness Score', 'Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)',
                       'Freedom', 'Trust (Government Corruption)', 'Generosity', 'Year']
format_2015_to_2019 = hp_2015[column_reorder_2015]
format_2015_to_2019.columns = hp_2019.columns

# 2016
column_reorder_2016 = ['Happiness Rank', 'Country', 'Happiness Score', 'Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)',
                       'Freedom', 'Trust (Government Corruption)', 'Generosity', 'Year']
format_2016_to_2019 = hp_2016[column_reorder_2016]
format_2016_to_2019.columns = hp_2019.columns

# 2017
column_reorder_2017 = ['Happiness.Rank', 'Country', 'Happiness.Score', 'Economy..GDP.per.Capita.', 'Family',  'Health..Life.Expectancy.',
                       'Freedom', 'Generosity', 'Trust..Government.Corruption.', 'Year']
format_2017_to_2019 = hp_2017[column_reorder_2017]
format_2017_to_2019.columns = hp_2019.columns

In [219]:
# Aggregating 
happiness_15_19 = hp_2019.append(hp_2018, ignore_index=True).append(format_2017_to_2019, ignore_index=True).append(format_2016_to_2019, ignore_index=True)
happiness_15_19 = happiness_15_19.append(format_2015_to_2019, ignore_index=True)

In [220]:
# Checking for countries not represented in all 5 years of data
country_counts = happiness_15_19.groupby('Country or region').count()
sparse_countries = country_counts[country_counts['Overall rank'] < 5]['Overall rank'].sort_values(ascending=False)
sparse_countries

Country or region
Angola                      4
Taiwan                      4
Central African Republic    4
Sudan                       4
South Sudan                 4
Hong Kong                   4
Laos                        4
Somalia                     4
Lesotho                     4
Macedonia                   4
Mozambique                  4
Namibia                     4
North Cyprus                3
Belize                      3
Comoros                     3
Trinidad and Tobago         3
Trinidad & Tobago           2
Northern Cyprus             2
Suriname                    2
Swaziland                   2
Oman                        1
Puerto Rico                 1
Somaliland Region           1
Hong Kong S.A.R., China     1
Somaliland region           1
Gambia                      1
Djibouti                    1
Taiwan Province of China    1
North Macedonia             1
Name: Overall rank, dtype: int64

We want to consolidate our data to represent only the countries that are present in all 5 files. Some of the above output represent the same country but are formatted differently:
- 'Taiwan Province of China' and 'Taiwan'
- 'Trinidad and Tobago' and 'Trinidad & Tobago'
- 'North Cyprus' and 'Northern Cyprus'
- 'Hong Kong' and 'Hong Kong S.A.R., China'
- 'Somalia' and 'Somaliland region'
- 'Macedonia' and 'North Macedonia'

We will to fix these 6 country formats manually. Then we will remove any other countries that are not represented in all 5 files.

In [221]:
# Replacing 'Taiwan Province of China' with 'Taiwan'
happiness_15_19['Country or region'] = ['Taiwan' if country=='Taiwan Province of China' else country for country in happiness_15_19['Country or region']]

# Replacing 'Trinidad & Tobago' with 'Trinidad and Tobago'
happiness_15_19['Country or region'] = ['Trinidad and Tobago' if country=='Trinidad & Tobago' else country for country in happiness_15_19['Country or region']]

# Replacing 'Northern Cyprus' with 'North Cyprus'
happiness_15_19['Country or region'] = ['North Cyprus' if country=='Northern Cyprus' else country for country in happiness_15_19['Country or region']]

# Replacing 'Hong Kong S.A.R., China' with 'Hong Kong'
happiness_15_19['Country or region'] = ['Hong Kong' if country=='Hong Kong S.A.R., China' else country for country in happiness_15_19['Country or region']]

# Replacing 'Somaliland region' with 'Somalia'
happiness_15_19['Country or region'] = ['Somalia' if country=='Somaliland region' else country for country in happiness_15_19['Country or region']]

# Replacing 'Noth Macedonia' with 'Macedonia'
happiness_15_19['Country or region'] = ['Macedonia' if country=='North Macedonia' else country for country in happiness_15_19['Country or region']]

# Making a list of countries not represented in all years
country_counts = happiness_15_19.groupby('Country or region').count()
remaining_sparse_countries = country_counts[country_counts['Overall rank'] < 5]['Overall rank'].sort_values(ascending=False)

# Removing countries not represented in all years
mask_remove_sparse = [False if country in remaining_sparse_countries.index.values else True for country in happiness_15_19['Country or region']]
happiness_15_19 = happiness_15_19[mask_remove_sparse]

We have combined the five world happiness report datasets into a cohesive and data-dense set, dropping around 15 countries in aggregation that were not represented by all 5 reports.

## 1.2 Combining the World Development Indicators and World Happiness Reports 

We begin by creating a comprehensive list of the countries in WDI dataset and WHR, fixing the format of both the WDI and the WHR datasets. Then, we reformat each of the files and aggregate them into a single, cohesive file.

### 1.2.1 Finalizing List of Countries and Their Rank

We will create a list of all countries that are represented in both the 'happiness_15_19' dataframe and the world development indicators dataset. We then query our WDI and happiness dataframes to contain only the countries in our country list.

In [222]:
# In happiness and not in wdi
missing_from_happiness_data = []
for country in happiness_15_19['Country or region'].unique():
  if country not in wdi_data['Country Name'].unique():
    missing_from_happiness_data.append(country)


# In wdi and not in happiness
missing_from_wdi_data = []
for country in wdi_data['Country Name'].unique():
  if country not in happiness_15_19['Country or region'].unique():
    missing_from_wdi_data.append(country)

print("The number of missing countries from aggregated happiness dataset is", len(missing_from_happiness_data))
print(missing_from_happiness_data)

print('\n\nThe number of missing countries from wdi data is', len(missing_from_wdi_data))
print(missing_from_wdi_data)

The number of missing countries from aggregated happiness dataset is 17
['Taiwan', 'Slovakia', 'South Korea', 'North Cyprus', 'Russia', 'Hong Kong', 'Macedonia', 'Kyrgyzstan', 'Ivory Coast', 'Congo (Brazzaville)', 'Venezuela', 'Palestinian Territories', 'Iran', 'Congo (Kinshasa)', 'Egypt', 'Syria', 'Yemen']


The number of missing countries from wdi data is 90
['American Samoa', 'Andorra', 'Angola', 'Antigua and Barbuda', 'Aruba', 'Bahamas, The', 'Barbados', 'Belize', 'Bermuda', 'British Virgin Islands', 'Brunei Darussalam', 'Cabo Verde', 'Cayman Islands', 'Central African Republic', 'Channel Islands', 'Comoros', 'Congo, Dem. Rep.', 'Congo, Rep.', "Cote d'Ivoire", 'Cuba', 'Curacao', 'Djibouti', 'Dominica', 'Egypt, Arab Rep.', 'Equatorial Guinea', 'Eritrea', 'Eswatini', 'Faroe Islands', 'Fiji', 'French Polynesia', 'Gambia, The', 'Gibraltar', 'Greenland', 'Grenada', 'Guam', 'Guinea-Bissau', 'Guyana', 'Hong Kong SAR, China', 'Iran, Islamic Rep.', 'Isle of Man', 'Kiribati', "Korea, Dem. Pe

To maximize the number of countries in our final dataset, we will manually match countries that are formatted differently.

This includes 14 matching countries:
- 'Slovakia' and 'Slovak Republic'
- 'South Korea' and 'Korea, Rep.' (Not to be confused with North Korea)
- 'Russia' and 'Russian Federation'
- 'Hong Kong' and 'Hong Kong SAR, China'
- 'Macedonia' and 'North Macedonia'
- 'Kyrgyzstan' and 'Kyrgyz Republic'
- 'Ivory Coast' and 'Cote d'Ivoire'
- 'Congo (Brazzaville)' and 'Congo, Rep.'
- 'Venezuela' and 'Venezuela, RB'
- 'Iran' and 'Iran, Islamic Rep.'
- 'Congo (Kinshasa)' and 'Congo, Dem. Rep.'
- 'Egypt' and 'Egypt, Arab Rep.'
- 'Syria' and 'Syrian Arab Republic'
- 'Yemen' and 'Yemen, Rep.'

In [223]:
# Creating a dictionary to match wdi country format to happiness country format 
match_happiness_wdi = {'Slovakia' : 'Slovak Republic', 'South Korea' : 'Korea, Rep.', 'Russia' : 'Russian Federation', 'Hong Kong' : 'Hong Kong SAR, China',
                       'Macedonia' : 'North Macedonia', 'Kyrgyzstan' : 'Kyrgyz Republic', 'Ivory Coast' : '''Cote d'Ivoire''', 'Congo (Brazzaville)' : 'Congo, Rep.',
                       'Venezuela' : 'Venezuela, RB', 'Iran' : 'Iran, Islamic Rep.', 'Congo (Kinshasa)' : 'Congo, Dem. Rep.', 'Egypt' : 'Egypt, Arab Rep.', 
                       'Syria' : 'Syrian Arab Republic', 'Yemen' : 'Yemen, Rep.'}

# Reverse (key, value) pair order
match_happiness_wdi = dict(map(reversed, match_happiness_wdi.items()))

# Change wdi country format to happiness country format  
wdi_data = wdi_data.replace({'Country Name': match_happiness_wdi})

# Creating a final list of countries represented in both happiness and wdi dataframes
final_country_set = set(wdi_data['Country Name'].unique()) & set(happiness_15_19['Country or region'].unique())
len(final_country_set)

144

In [224]:
# Querying the wdi data for the matching 144 countries
wdi_data = wdi_data[wdi_data['Country Name'].isin(final_country_set)]

# Querying the happiness data for the matching 144 countries
happiness_15_19 = happiness_15_19[happiness_15_19['Country or region'].isin(final_country_set)]

# Check if country formats between the two datasets match
(wdi_data['Country Name'].sort_values().unique() == happiness_15_19['Country or region'].sort_values().unique()).all()

True

Next, we will fix the happiness rankings in the 'happiness_15_19' dataframe, because the **'Overall rank' field represents the wrong list of countries**, and we want it to represent the 144 countries in the 'final_country_set' object.

In [226]:
# Here we see that rankings go from 1 to 158, when there are only 144 unique countries
print('Number of Unique Rankings is:', len(happiness_15_19['Overall rank'].unique()))

Number of Unique Rankings is: 158


In [227]:
# Change the rank to be from 1 to 144 for each year
happiness_temp = pd.DataFrame(columns=happiness_15_19.columns)

for year in happiness_15_19['Year'].unique():
  temp_year_df = happiness_15_19[happiness_15_19['Year']==year].sort_values(by='Overall rank')
  temp_year_df['Overall rank'] = range(1,len(temp_year_df['Overall rank'])+1)
  happiness_temp = pd.concat([happiness_temp, temp_year_df], axis=0)
  
happiness_15_19 = happiness_temp

# Now the rankings go from 1 to 144
print('Number of Unique Rankings is:', len(happiness_15_19['Overall rank'].unique()))

Number of Unique Rankings is: 144


We see that the countries in the two dataframes 'wdi_data' and 'happiness_15_19' have the same remaining countries, and 'happiness_15_19' has the country ranks ordered correctly.

### 1.2.2 Reformat Happiness Data to Match WDI Fields
The two dataframes 'wdi_data' and 'happiness_15_19' have different columns. Here we will reformat 'happiness_15_19' to have the same column format of 'wdi_data'.

In [228]:
# This is our desired format
wdi_data.head(2)

,Country Name,Country Code,Series Name,Series Code,2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019]
0,United States,USA,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,100,100,..,..,..
1,United States,USA,Access to electricity (% of population),EG.ELC.ACCS.ZS,100,100,100,100,..


In [229]:
# This is our current format
happiness_15_19.head(2)

,Overall rank,Country or region,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Year
0,1,Finland,7.769,1.340,1.587,0.986,0.596,0.153,0.393,2019
1,2,Denmark,7.600,1.383,1.573,0.996,0.592,0.252,0.410,2019


In [230]:
# Changing indicies and columsn in 'happiness_15_19' to look more like 'wdi_data'
happ_interim = happiness_15_19.set_index(['Country or region', 'Year']).unstack().stack(level = 0).reset_index()
happ_interim.head()

# Add 'Series Code' column
happ_interim['Series Code'] = 'HAPPINESS.REPORT.' + happ_interim['level_1'].str.split().str[0].str.upper()

# Add 'Country Code' column
country_code_dict = pd.Series(wdi_data['Country Code'].values,index=wdi_data['Country Name'].values).to_dict()
happ_interim['Country Code'] = happ_interim['Country or region'].map(country_code_dict)

# Reorder columns to match wdi
order = ['Country or region', 'Country Code', 'level_1', 'Series Code', 2015, 2016, 2017, 2018, 2019]
happ_interim = happ_interim[order]

# Add the same columns
happ_interim.columns = wdi_data.columns

# Output format matches that of wdi_data
happ_interim.head()

,Country Name,Country Code,Series Name,Series Code,2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019]
0,Afghanistan,AFG,Freedom to make life choices,HAPPINESS.REPORT.FREEDOM,0.23414,0.1643,0.10618,0.085,0
1,Afghanistan,AFG,GDP per capita,HAPPINESS.REPORT.GDP,0.31982,0.38227,0.401477,0.332,0.35
2,Afghanistan,AFG,Generosity,HAPPINESS.REPORT.GENEROSITY,0.09719,0.07112,0.311871,0.191,0.158
3,Afghanistan,AFG,Healthy life expectancy,HAPPINESS.REPORT.HEALTHY,0.30335,0.17344,0.180747,0.255,0.361
4,Afghanistan,AFG,Overall rank,HAPPINESS.REPORT.OVERALL,139,141,132,135,144


### 1.2.3 Aggregating and Cleaning The Final Dataset

In [231]:
# Aggregating the two datasets
wdi_happiness = pd.concat([wdi_data, happ_interim], axis=0, ignore_index=True)#.sort_values(by=['Country Name','Series Name'])

# Fixing format of year column titles
fix_year_dict = {'2015 [YR2015]':'2015', '2016 [YR2016]':'2016', '2017 [YR2017]':'2017', '2018 [YR2018]':'2018', '2019 [YR2019]':'2019'}
wdi_happiness = wdi_happiness.rename(columns = fix_year_dict)

# Final check that wdi_happiness contains the happiness report data and printing output
print('''Contains 'overall rank' field:''', wdi_happiness['Series Code'].str.contains('HAPPINESS.REPORT.OVERALL').any())
wdi_happiness.head()

Contains 'overall rank' field: True


,Country Name,Country Code,Series Name,Series Code,2015,2016,2017,2018,2019
0,United States,USA,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,100,100,..,..,..
1,United States,USA,Access to electricity (% of population),EG.ELC.ACCS.ZS,100,100,100,100,..
2,United States,USA,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,100,100,100,100,..
3,United States,USA,"Access to electricity, urban (% of urban popul...",EG.ELC.ACCS.UR.ZS,100,100,100,100,..
4,United States,USA,Account ownership at a financial institution o...,FX.OWN.TOTL.ZS,..,..,93.1222,..,..


## 1.3 Outputting the Data
Now, we will output 3 files as CSVs
- wdi_happiness (both wdi data and happiness data)
- wdi_formatted (wdi data only)
- happiness_formatted (happiness data only)

In [232]:
wdi_happiness.to_csv(r'wdi_happiness.csv', index = False, header=True)
wdi_data.to_csv(r'wdi_formatted.csv', index = False, header=True)
happ_interim.to_csv(r'happiness_formatted.csv', index = False, header=True)